# Обработка текстов

Вам дан файл `ru.conversations.txt`

В нем содержатся диалоги (разделены пустой строкой).

Вам необходимо:

### Разбить текст на диалоги

In [1]:
with open('ru.conversations.txt', 'r', encoding='utf-8') as f:
    text = []
    for line in f:
        text.append(line)

In [2]:
text[:10]

['- Что,  Мансур,  не жарко теперь тебе?\n',
 '- Спрашиваю, не жарко ему теперь?\n',
 '\n',
 '- Какой полк?\n',
 '- Тысяча тридцать четвертый.\n',
 '- Вези дальше. Тут тысяча двадцать шестой.\n',
 '\n',
 '- Какая санрота?\n',
 '- Тысяча тридцать шестая.\n',
 '- Значит, наша! Принимай тяжелораненого!\n']

In [3]:
dialogs = []
dialog = ''
for item in text:
    if item != '\n':
        dialog += item
    else:
        dialogs.append(dialog)
        dialog = ''
print(dialogs[:5])

['- Что,  Мансур,  не жарко теперь тебе?\n- Спрашиваю, не жарко ему теперь?\n', '- Какой полк?\n- Тысяча тридцать четвертый.\n- Вези дальше. Тут тысяча двадцать шестой.\n', '- Какая санрота?\n- Тысяча тридцать шестая.\n- Значит, наша! Принимай тяжелораненого!\n', '- Сколько фрицев в котле?\n- Тысяч сорок.\n', '- Чем же он отравился?\n- А вон, видишь, что-то из тех бутылей выпил.\n']


### Разбить диалоги на токены

In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
for i,item in enumerate(dialogs):
    dialogs[i] = word_tokenize(item)

### Предобработать токены
Удалить стоп-слова и спецсимволы, лемматизировать или стеммировать слова и т.д.

In [7]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
tokenizer = RegexpTokenizer(r'\w+')
for i,item in enumerate(dialogs):
    dialogs[i] = tokenizer.tokenize(str(dialogs[i]))

In [10]:
clean_dialogs = dialogs
for i, dialog in enumerate(dialogs):
    clean_dialog = dialog
    for word in dialog:
        if word.lower() in stopwords.words('russian'):
            clean_dialog.remove(word)
    clean_dialogs[i] = clean_dialog

In [11]:
clean_dialogs[:5]

[['Мансур', 'жарко', 'тебе', 'Спрашиваю', 'жарко', 'теперь'],
 ['полк',
  'Тысяча',
  'тридцать',
  'четвертый',
  'Вези',
  'дальше',
  'тысяча',
  'двадцать',
  'шестой'],
 ['санрота',
  'Тысяча',
  'тридцать',
  'шестая',
  'Значит',
  'наша',
  'Принимай',
  'тяжелораненого'],
 ['Сколько', 'фрицев', 'котле', 'Тысяч', 'сорок'],
 ['же', 'отравился', 'вон', 'видишь', 'то', 'тех', 'бутылей', 'выпил']]

In [12]:
from nltk.stem import SnowballStemmer
rus_stemmer = SnowballStemmer('russian')

In [13]:
%%timeit
rus_stemmer.stem('котле')

75.9 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
stem_dialogs = clean_dialogs
for i, dialog in enumerate(clean_dialogs):
    stem_dialog = dialog
    for j, word in enumerate(dialog):
        stem_dialog[j] = rus_stemmer.stem(word)
    stem_dialogs[i] = stem_dialog

### Перевести каждый из диалогов в векторное представление

С помощью TF-IDF представить каждый из диалогов в векторном формате

In [19]:
import sklearn.feature_extraction.text as txt

In [49]:
stem_dialogs[1]

['полк',
 'тысяч',
 'тридца',
 'четверт',
 'вез',
 'дальш',
 'тысяч',
 'двадца',
 'шест']

In [50]:
stings = [' '.join(map(str, stem_dialogs[i])) for i in range(len(stem_dialogs))]

In [58]:
tfvec = txt.TfidfVectorizer(ngram_range=(1, 1))
vect = tfvec.fit_transform(stings)

In [43]:
print(two_sent_embed[1], stings[1])

  (0, 23802)	0.3358460324460381
  (0, 33034)	0.5753375355523771
  (0, 32733)	0.3055167923759493
  (0, 36132)	0.31814876630578093
  (0, 4203)	0.3676099365329098
  (0, 7622)	0.25718876924659617
  (0, 7742)	0.28201226211047364
  (0, 36672)	0.28429029231441805 полк тысяч тридца четверт вез дальш тысяч двадца шест


In [54]:
tfvec.vocabulary_

{'мансур': 16297,
 'жарк': 9680,
 'теб': 31962,
 'спрашива': 30731,
 'тепер': 32084,
 'полк': 23802,
 'тысяч': 33034,
 'тридца': 32733,
 'четверт': 36132,
 'вез': 4203,
 'дальш': 7622,
 'двадца': 7742,
 'шест': 36672,
 'санрот': 28614,
 'знач': 11204,
 'наш': 18624,
 'принима': 25447,
 'тяжелоранен': 33108,
 'скольк': 29658,
 'фриц': 34787,
 'котл': 14148,
 'сорок': 30407,
 'же': 9708,
 'отрав': 21183,
 'вон': 5139,
 'вид': 4585,
 'то': 32320,
 'тех': 32173,
 'бутыл': 3753,
 'вып': 5833,
 'что': 36293,
 'дел': 7942,
 'фашист': 34271,
 'прорва': 26027,
 'оборон': 19891,
 'мы': 17757,
 'тоб': 32321,
 'спим': 30631,
 'их': 12123,
 'гумрак': 7452,
 'комсорг': 13744,
 'не': 18650,
 'прот': 26114,
 'ещ': 9611,
 'сто': 31088,
 'брал': 3397,
 'во': 4918,
 'ты': 33024,
 'сбил': 28750,
 'может': 17273,
 'нибуд': 19228,
 'стреля': 31219,
 'где': 6371,
 'комбат': 13655,
 'унесл': 33634,
 'ран': 27081,
 'сынок': 31577,
 'лошад': 15699,
 'надежд': 18007,
 'да': 7537,
 'вы': 5576,
 'отец': 20975,
 'с

In [55]:
from scipy.spatial.distance import euclidean

In [74]:
vect[1]

<1x38090 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

### Найти самые близкие по содержанию диалоги, исходя из векторного представления